<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/model/MIMIC/MIMIC_simcse_progress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/danielsaggau/IR_LDC.git

fatal: destination path 'IR_LDC' already exists and is not an empty directory.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [9]:
%cd IR_LDC

/content/IR_LDC


In [ ]:
!pip install -r requirements.txt

In [6]:
from datasets import load_dataset
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import LoggingHandler, SentenceTransformer, InputExample
import logging
from datetime import datetime
import gzip
import sys
import tqdm
import json

In [ ]:
!unzip /content/drive/MyDrive/mimic.jsonl.zip -d content
with open('content/mimic.jsonl') as f:
    data = [json.loads(line) for line in f]

In [ ]:
dataset = load_dataset("json", data_files="/content/IR_LDC/content/mimic.jsonl")

Todo: learn how to work with mimic script
``python /content/IR_LDC/model/MIMIC/mimic.py``

In [8]:
dataset_train_test = dataset['train'].train_test_split(test_size=0.1)
dataset_test = dataset_train_test['test']
dataset_sp = dataset_train_test['train'].train_test_split(test_size=0.1/0.9)
dataset_train = dataset_sp['train']
dataset_validation = dataset_sp['test']

In [13]:
dataset_train_test

DatasetDict({
    train: Dataset({
        features: ['summary_id', 'concepts', 'text', 'sections', 'level_1', 'level_2', 'level_3', 'data_type'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['summary_id', 'concepts', 'text', 'sections', 'level_1', 'level_2', 'level_3', 'data_type'],
        num_rows: 5000
    })
})

In [9]:
dataset_train_test['test']['text'][9]

"Admission Date:  [**2142-8-17**]              Discharge Date:   [**2142-9-21**]\n\nDate of Birth:  [**2089-7-6**]             Sex:   F\n\nService: MEDICINE\n\nAllergies:\nTape\n\nAttending:[**First Name3 (LF) 297**]\nChief Complaint:\nInfected lower extremity ulcer\n\nMajor Surgical or Invasive Procedure:\nPercutaneous tracheostomy placement\n\n\nHistory of Present Illness:\nCC: Respiratory failure\n.\nHPI: The patient was initially admitted to OSH for treatment of\nRLE ulcers from venous stasis/arterial insufficiency. The\npatient was intended to have fem-[**Doctor Last Name **] bypass, but the ulcers\nbecame infected with MRSA and the patient was taken to the OR at\nthe OSH for debridement. The patient was then stabilized and\ntransferred to rehab on a regimen of vanc/zosyn/cipro. An MRI\nshowed no evidence of osteo, but MRA showed minimal inflow to\nRLE with SFA occlusion. At the rehab center, the patient\nexperienced increasing pain and erythema at the RLE site, and\nwas then tran

In [14]:
with open('your_file.txt', 'w') as f:
    for line in dataset_train_test['test']['text']:
        f.write(f"{line}\n")

In [ ]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

# Training parameters
#access="XXXX"
model_name = "bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12"
train_batch_size = 6
max_seq_length = 4096
num_epochs = 1

#Input file path (a text file, each line a sentence)
if len(sys.argv) < 2:
    print("Run this script with: python {} path/to/sentences.txt".format(sys.argv[0]))
    exit()

#filepath = sys.argv[1]


# Save path to store our model
output_name = ''
if len(sys.argv) >= 3:
    output_name = "-"+sys.argv[2].replace(" ", "_").replace("/", "_").replace("\\", "_")

model_output_path = '/content/drive/MyDrive/output/train_simcse{}-{}'.format(output_name, datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

# Use Huggingface/transformers model (like BERT, RoBERTa, XLNet, XLM-R) for mapping tokens to embeddings
model = SentenceTransformer(model_name)


In [ ]:
################# Read the train corpus  #################
filepath = "your_file.txt"
train_samples = []
with gzip.open(filepath, 'rt', encoding='utf8') if filepath.endswith('.gz') else open(filepath, encoding='utf8') as fIn:
    for line in tqdm.tqdm(fIn, desc='Read file'):
        line = line.strip()
        if len(line) >= 10:
            train_samples.append(InputExample(texts=[line, line]))

logging.info("Train sentences: {}".format(len(train_samples)))

# We train our model using the MultipleNegativesRankingLoss
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size, drop_last=True)
train_loss = losses.MultipleNegativesRankingLoss(model)
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          optimizer_params={'lr': 3e-5},
          checkpoint_path=model_output_path,
          show_progress_bar=True,
          use_amp=True  # Set to True, if your GPU supports FP16 cores
          )

Read file: 1385355it [00:04, 278866.79it/s]


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/180139 [00:00<?, ?it/s]